In [ ]:
# ============================================================
# Cell 1: 環境セットアップ
# ============================================================
!pip install -q transformers accelerate peft bitsandbytes datasets sentencepiece

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# ============================================================
# Cell 2: Google Driveマウント＆データ読み込み
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# 📁 データファイルのパスを設定（必要に応じて変更）
DATA_PATH = "/content/drive/MyDrive/noteAI/"
TRAINING_FILE = DATA_PATH + "generation_training.jsonl"

import json

# 学習データ読み込み
training_data = []
with open(TRAINING_FILE, "r", encoding="utf-8") as f:
    for line in f:
        training_data.append(json.loads(line))

print(f"✅ 学習データ: {len(training_data)} 件")
print(f"\n📝 サンプル:")
print(json.dumps(training_data[0], ensure_ascii=False, indent=2))

In [ ]:
# ============================================================
# Cell 3: データセット準備（Alpaca形式）
# ============================================================
from datasets import Dataset

def format_for_training(example):
    """Alpaca形式に変換"""
    text = f"""### 指示:\n{example['instruction']}\n\n### 入力:\n{example['input'] if example['input'] else '（なし）'}\n\n### 応答:\n{example['output']}"""
    return {"text": text}

# Dataset作成
dataset = Dataset.from_list(training_data)
dataset = dataset.map(format_for_training)

print(f"✅ データセットサイズ: {len(dataset)}")
print(f"\n📄 フォーマット後のサンプル:")
print("=" * 50)
print(dataset[0]["text"])
print("=" * 50)

In [ ]:
# ============================================================
# Cell 4: モデル読み込み（4bit量子化）
# ============================================================
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 🤖 日本語対応モデル
MODEL_NAME = "rinna/japanese-gpt-neox-3.6b-instruction-sft"

# 量子化設定（GPUメモリ節約）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# トークナイザー
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# モデル読み込み
print(f"🔄 モデル読み込み中... ({MODEL_NAME})")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ モデル読み込み完了")
print(f"📊 パラメータ数: {model.num_parameters() / 1e9:.2f}B")

In [ ]:
# ============================================================
# Cell 5: LoRA設定
# ============================================================
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# モデルをLoRA学習用に準備
model = prepare_model_for_kbit_training(model)

# LoRA設定
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],  # GPT-NeoX用
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA適用
model = get_peft_model(model, lora_config)

print("✅ LoRA設定完了")
model.print_trainable_parameters()

In [ ]:
# ============================================================
# Cell 6: トレーニング設定
# ============================================================
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# データのトークナイズ
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names
)

# トレーニング設定
training_args = TrainingArguments(
    output_dir=DATA_PATH + "checkpoints",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_strategy="epoch",
    optim="paged_adamw_32bit",
    report_to="none",
)

# データコレクター
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("✅ トレーニング準備完了！")
print(f"📊 エポック数: {training_args.num_train_epochs}")
print(f"📊 バッチサイズ: {training_args.per_device_train_batch_size}")
print(f"📊 学習率: {training_args.learning_rate}")

In [ ]:
# ============================================================
# Cell 7: 学習実行 🚀
# ============================================================
print("🚀 学習開始...")
trainer.train()
print("\n✅ 学習完了！")

# モデル保存
MODEL_OUTPUT = DATA_PATH + "note_title_model"
model.save_pretrained(MODEL_OUTPUT)
tokenizer.save_pretrained(MODEL_OUTPUT)
print(f"💾 モデル保存先: {MODEL_OUTPUT}")

In [ ]:
# ============================================================
# Cell 8: タイトル生成テスト 🎉
# ============================================================

def generate_titles(keyword: str, num_titles: int = 5):
    """キーワードからタイトル候補を生成"""
    prompt = f"""### 指示:
「{keyword}」に関する、読者の興味を引くnote記事タイトルを生成してください。

### 入力:
（なし）

### 応答:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    titles = []
    for i in range(num_titles):
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2
        )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        title = generated.split("### 応答:")[-1].strip()
        # 改行以降を削除
        title = title.split("\n")[0].strip()
        if title and title not in titles:
            titles.append(title)

    return titles

# テスト実行
print("=" * 60)
print("🎯 タイトル生成テスト")
print("=" * 60)

test_keywords = ["副業で月5万円", "AI活用術", "30代の転職", "子育ての悩み"]

for kw in test_keywords:
    print(f"\n📝 【{kw}】のタイトル案:")
    print("-" * 40)
    for i, title in enumerate(generate_titles(kw), 1):
        print(f"  {i}. {title}")

In [ ]:
# ============================================================
# Cell 9: インタラクティブ生成
# ============================================================

# 自由にキーワードを入力してタイトル生成
keyword = input("キーワードを入力: ")
print(f"\n🎯 【{keyword}】のタイトル案:")
for i, title in enumerate(generate_titles(keyword, 10), 1):
    print(f"  {i}. {title}")

---
## 📦 モデルのダウンロード

学習したモデルはGoogle Drive上に保存されています:
- `MyDrive/noteAI/note_title_model/`

ローカルで使用する場合は、上記フォルダをダウンロードしてください。